# Ratings_cleaning musi zostać uruchomiony jako pierwszy
Notebook `Ratings_cleaning.ipynb` usuwa niektóre z wystawionych ocen, co zmniejsza ich liczbę u użytkowników. Następnie usuwa użytkowników, którzy mają mniej niż 20 ocen i zapisuje nowy zbiór danych do pliku `user_features_2.csv`.

In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('../data/user_features_train_2.csv')
df_test = pd.read_csv('../data/user_features_test_2.csv')

In [3]:
print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159120 entries, 0 to 159119
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   userId                 159120 non-null  int64  
 1   num_rating             159120 non-null  int64  
 2   avg_rating             159120 non-null  float64
 3   weekend_watcher        159120 non-null  int64  
 4   type_of_viewer         159120 non-null  object 
 5   genre_Action           158184 non-null  float64
 6   genre_Adventure        158377 non-null  float64
 7   genre_Animation        138044 non-null  float64
 8   genre_Comedy           158785 non-null  float64
 9   genre_Crime            157465 non-null  float64
 10  genre_Documentary      58396 non-null   float64
 11  genre_Drama            159071 non-null  float64
 12  genre_Family           149620 non-null  float64
 13  genre_Fantasy          155643 non-null  float64
 14  genre_History          138167 non-nu

# Uzupełnienie brakujących ocen 

In [4]:
genre_columns = ['genre_Action','genre_Adventure','genre_Animation','genre_Comedy','genre_Crime','genre_Documentary','genre_Drama','genre_Family','genre_Fantasy','genre_History','genre_Horror','genre_Musical','genre_Mystery','genre_Romance','genre_Science Fiction','genre_TV Movie','genre_Thriller','genre_War','genre_Western']

for col in genre_columns:
    df_train[col] = df_train[col].fillna(df_train['avg_rating'])
    df_test[col] = df_test[col].fillna(df_test['avg_rating'])

print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159120 entries, 0 to 159119
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   userId                 159120 non-null  int64  
 1   num_rating             159120 non-null  int64  
 2   avg_rating             159120 non-null  float64
 3   weekend_watcher        159120 non-null  int64  
 4   type_of_viewer         159120 non-null  object 
 5   genre_Action           159120 non-null  float64
 6   genre_Adventure        159120 non-null  float64
 7   genre_Animation        159120 non-null  float64
 8   genre_Comedy           159120 non-null  float64
 9   genre_Crime            159120 non-null  float64
 10  genre_Documentary      159120 non-null  float64
 11  genre_Drama            159120 non-null  float64
 12  genre_Family           159120 non-null  float64
 13  genre_Fantasy          159120 non-null  float64
 14  genre_History          159120 non-nu

# Enkodowanie kolumny 'type_of_viewer'

In [5]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)

def encoder(df):
    encoded = ohe.fit_transform(df[['type_of_viewer']])
    encoded = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(['type_of_viewer']))
    encoded.index = df.index
    df = df.drop(columns=['type_of_viewer'])
    df = pd.concat([df, encoded], axis=1)

    return df

df_train = encoder(df_train)
df_test = encoder(df_test)

print(df_train.head(3))
print(df_test.head(3))


   userId  num_rating  avg_rating  weekend_watcher  genre_Action  \
0       1         141    3.531915                0      4.133333   
1       2          52    4.269231                0      4.000000   
2       3         147    3.588435                0      3.583333   

   genre_Adventure  genre_Animation  genre_Comedy  genre_Crime  \
0         3.333333         3.531915      3.339623     4.285714   
1         4.142857         4.875000      4.130435     4.000000   
2         3.605263         3.961538      3.186275     3.346154   

   genre_Documentary  ...  genre_Mystery  genre_Romance  \
0           3.000000  ...       4.272727       3.487805   
1           4.269231  ...       4.750000       4.823529   
2           3.588435  ...       3.875000       3.637931   

   genre_Science Fiction  genre_TV Movie  genre_Thriller  genre_War  \
0               4.000000        3.531915        3.666667   3.555556   
1               4.269231        4.269231        4.300000   3.000000   
2           

# Dołączenie ostatnich 20 ocenionych filmów ze zbioru ratings_clean_zlib.h5

In [6]:
ratings_df = pd.read_parquet('../data/ratings_clean_groupped_20.parquet')
df_train = pd.merge(df_train, ratings_df, on='userId')
df_test = pd.merge(df_test, ratings_df, on='userId')

# Normalizacja danych numerycznych

In [7]:
from sklearn.preprocessing import StandardScaler

columns_to_normalize = ['num_rating', 'avg_rating']
columns_to_normalize.extend(genre_columns)
print('Columns to normalize:', columns_to_normalize)

scaler = StandardScaler()

df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])
print(df_train.head(3))

Columns to normalize: ['num_rating', 'avg_rating', 'genre_Action', 'genre_Adventure', 'genre_Animation', 'genre_Comedy', 'genre_Crime', 'genre_Documentary', 'genre_Drama', 'genre_Family', 'genre_Fantasy', 'genre_History', 'genre_Horror', 'genre_Musical', 'genre_Mystery', 'genre_Romance', 'genre_Science Fiction', 'genre_TV Movie', 'genre_Thriller', 'genre_War', 'genre_Western']
   userId  num_rating  avg_rating  weekend_watcher  genre_Action  \
0       1   -0.067753   -0.348394                0      0.927204   
1       2   -0.379224    1.210416                0      0.713512   
2       3   -0.046755   -0.228900                0      0.045723   

   genre_Adventure  genre_Animation  genre_Comedy  genre_Crime  \
0        -0.374819         -0.18135     -0.403073     0.892212   
1         0.941074          1.58054      0.972506     0.410036   
2         0.067207          0.38224     -0.669814    -0.693406   

   genre_Documentary  ...  genre_TV Movie  genre_Thriller  genre_War  \
0         

## Dla danych testowych

In [8]:
df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])
print(df_test.head(3))

   userId  num_rating  avg_rating  weekend_watcher  genre_Action  \
0       5   -0.445718   -0.896360                0      0.179281   
1       7   -0.407221   -0.127572                1     -0.889181   
2      13   -0.333728   -0.106873                0      0.123046   

   genre_Adventure  genre_Animation  genre_Comedy  genre_Crime  \
0        -0.374819        -0.004580     -0.993829    -1.910438   
1        -0.510278        -0.223217      0.000142     0.035010   
2        -0.555431        -0.031489     -0.191006    -0.795405   

   genre_Documentary  ...  genre_TV Movie  genre_Thriller  genre_War  \
0          -0.682450  ...       -0.744344       -0.373172   0.196501   
1           1.846838  ...       -0.100974       -0.076176  -0.315053   
2          -0.135632  ...       -0.083653       -0.252328   0.196501   

   genre_Western  type_of_viewer_negative  type_of_viewer_neutral  \
0      -0.911831                      0.0                     1.0   
1      -0.241608                   

# Konwersja kolumny weekend_watcher do float64

In [9]:
df_train['weekend_watcher'] = df_train['weekend_watcher'].astype('float64')
df_test['weekend_watcher'] = df_test['weekend_watcher'].astype('float64')

In [10]:
print('\', \''.join(df_train.columns.to_list()))

userId', 'num_rating', 'avg_rating', 'weekend_watcher', 'genre_Action', 'genre_Adventure', 'genre_Animation', 'genre_Comedy', 'genre_Crime', 'genre_Documentary', 'genre_Drama', 'genre_Family', 'genre_Fantasy', 'genre_History', 'genre_Horror', 'genre_Musical', 'genre_Mystery', 'genre_Romance', 'genre_Science Fiction', 'genre_TV Movie', 'genre_Thriller', 'genre_War', 'genre_Western', 'type_of_viewer_negative', 'type_of_viewer_neutral', 'type_of_viewer_positive', 'movies_seq', 'ratings_seq', 'ts_seq


# Zapis do pliku

In [11]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159120 entries, 0 to 159119
Data columns (total 29 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   userId                   159120 non-null  int64  
 1   num_rating               159120 non-null  float64
 2   avg_rating               159120 non-null  float64
 3   weekend_watcher          159120 non-null  float64
 4   genre_Action             159120 non-null  float64
 5   genre_Adventure          159120 non-null  float64
 6   genre_Animation          159120 non-null  float64
 7   genre_Comedy             159120 non-null  float64
 8   genre_Crime              159120 non-null  float64
 9   genre_Documentary        159120 non-null  float64
 10  genre_Drama              159120 non-null  float64
 11  genre_Family             159120 non-null  float64
 12  genre_Fantasy            159120 non-null  float64
 13  genre_History            159120 non-null  float64
 14  genr

In [13]:
df_train.to_parquet('../data/user_features_clean_train.parquet', compression='brotli')

In [14]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39808 entries, 0 to 39807
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   userId                   39808 non-null  int64  
 1   num_rating               39808 non-null  float64
 2   avg_rating               39808 non-null  float64
 3   weekend_watcher          39808 non-null  float64
 4   genre_Action             39808 non-null  float64
 5   genre_Adventure          39808 non-null  float64
 6   genre_Animation          39808 non-null  float64
 7   genre_Comedy             39808 non-null  float64
 8   genre_Crime              39808 non-null  float64
 9   genre_Documentary        39808 non-null  float64
 10  genre_Drama              39808 non-null  float64
 11  genre_Family             39808 non-null  float64
 12  genre_Fantasy            39808 non-null  float64
 13  genre_History            39808 non-null  float64
 14  genre_Horror          

In [16]:
df_test.to_parquet('../data/user_features_clean_train.parquet', compression='brotli')